In [1]:
X_COORD = 0
Y_COORD = 1
Z_COORD = 2

In [2]:
def print_data(sequence, boards, num_numbers, num_boards):
    num = min(num_numbers, len(sequence))
    print("First {} (from a total of {}) numbers: {}".format(num, len(sequence), sequence[:num]))
    num = min(num_boards, len(boards))
    print("First {} (from a total of {}) boards:".format(num, len(boards)))
    for board in boards[:num]:
        string = ""
        for row in board:
            string += "\t".join([str(num) for num in row]) + "\n"
        print(string)

In [3]:
def get_processed_input(input_path, sequence, boards):
    """The board is a 3D-matrix and the sequence is a list"""
    with open(input_path, "rt") as f:
        raw_input = f.read()
        sequence_end_i = raw_input.index("\n\n")
        sequence += [int(num) for num in raw_input[:sequence_end_i].split(",") if num]
        
        raw_boards = [board for board in raw_input[sequence_end_i:].split("\n\n") if board]
        for raw_board in raw_boards:
            raw_rows = [row for row in raw_board.split("\n") if row]
            board = []
            for raw_row in raw_rows:  # 0..num_columns-1
                board.append([int(num) for num in raw_row.split()])
            boards.append(board)    
        print_data(sequence, boards, 10, 3)

In [4]:
def index_nums_to_mats(sequence, boards, indices, num_boards, num_rows, num_cols):
    """Assuming that all the boards are of the same size"""
    # Maps a number with a list of 3D matrices. The format (X,Y,Z) of the indices represents:
    # X -> The matrix containing the number
    # Y -> The row of the matrix containing the number
    # Z -> The column of the matrix containing the number
    for x in range(num_boards):
        board = boards[x]
        for y in range(num_rows):
            for z in range(num_cols):
                num = board[y][z]
                mats_list = indices.get(num, [])
                mats_list.append((x,y,z))
                indices[num] = mats_list

In [5]:
def get_score(sequence, boards, board_id, pulled_nums_count, verbose=False):
    board = boards[board_id]
    score = 0
    all_numbers = [num for row in board for num in row]
    valid_numbers = [num for num in all_numbers if num not in sequence[:pulled_nums_count]]
    not_valid_numbers = [num for num in all_numbers if num in sequence[:pulled_nums_count]]   # Debug
    if verbose:
        print("Computed board numbers:\t\t{}\nNot computed numbers:\t\t{}".format(valid_numbers, not_valid_numbers))
    score = sum(valid_numbers) * sequence[pulled_nums_count-1]
    return score 

In [6]:
def part_a(sequence, boards, max_winners=1):
    num_to_mats = {}
    num_boards = len(boards)
    num_rows = len(boards[0])
    num_cols = len(boards[0][0])
    index_nums_to_mats(sequence, boards, num_to_mats, num_boards, num_rows, num_cols)
    
    # Maps a matrix index to the amount of numbers in its columns and rows that are still to be pulled out. Format (X,Y,Z) where:
    # X -> The matrix 
    # Y -> List of five counters representing the numbers left to be pulled out for each row of the matrix
    # Z -> List of five counters representing the numbers left to be pulled out for each column of the matrix
    left_to_win = {i:[[5 for j in range(num_rows)],[5 for j in range(num_cols)]] for i in range(num_boards)}
    winners = []
    pulled_nums_count = 0
    winner_found = False
    for pulled_num in sequence:
        pulled_nums_count += 1
        for mat_coords in num_to_mats.get(pulled_num):
            lucky_mat_id = mat_coords[X_COORD]
            lucky_mat_row = mat_coords[Y_COORD]
            lucky_mat_col = mat_coords[Z_COORD]
            lucky_mat = left_to_win.get(lucky_mat_id)
            lucky_mat[0][lucky_mat_row] -= 1
            lucky_mat[1][lucky_mat_col] -= 1
            left_to_win[lucky_mat_id] = lucky_mat
            if lucky_mat[0][lucky_mat_row] < 1 or lucky_mat[1][lucky_mat_col] < 1:
                winners.append(lucky_mat_id)
                winner_found = True
        if winner_found:
            break
    
    scores = []
    num_winners = min(max_winners, len(winners))
    for winner in winners[:num_winners]:
        scores.append((winner, get_score(sequence, boards, winner, pulled_nums_count)))
    return scores

In [19]:
def part_b(sequence, boards, max_losers=1, verbose=False):
    num_to_mats = {}
    num_boards = len(boards)
    num_rows = len(boards[0])
    num_cols = len(boards[0][0])
    index_nums_to_mats(sequence, boards, num_to_mats, num_boards, num_rows, num_cols)
    
    left_to_win = {i:[[5 for j in range(num_rows)],[5 for j in range(num_cols)]] for i in range(num_boards)}
    loser_candidates = [board_id for board_id in range(len(boards))]
    pulled_nums_count = 0
    loser_finished = loser_found = False
    all_removed_candidartes = set({})
    for pulled_num in sequence:
        removed_candidates = []
        pulled_nums_count += 1
        for mat_coords in num_to_mats.get(pulled_num):
            lucky_mat_id = mat_coords[X_COORD]
            lucky_mat_row = mat_coords[Y_COORD]
            lucky_mat_col = mat_coords[Z_COORD]
            lucky_mat = left_to_win.get(lucky_mat_id)
            lucky_mat[0][lucky_mat_row] -= 1
            lucky_mat[1][lucky_mat_col] -= 1
            left_to_win[lucky_mat_id] = lucky_mat
            if lucky_mat[0][lucky_mat_row] < 1 or lucky_mat[1][lucky_mat_col] < 1:
                if verbose:
                    print("{} board discarded when {} has been pulled out".format(lucky_mat_id, pulled_num))
                removed_candidates.append(lucky_mat_id)
                if loser_found:
                    loser_finished = True
                    break
                if lucky_mat_id not in all_removed_candidartes:
                    loser_candidates.remove(lucky_mat_id)
        for candidate in removed_candidates:
            all_removed_candidartes.add(candidate)
        if loser_finished:
            break
        if len(loser_candidates) < 2:
            if len(loser_candidates) < 1:
                loser_candidates += removed_candidates
            if not loser_found:
                if verbose:
                    print("Waiting for {} to end".format(loser_candidates))
                    # for mat_id in left_to_win.keys():
                    #     print("{} has emptied a row or a column: {}".format(mat_id, 0 in \
                    #               (left_to_win.get(mat_id)[0] + left_to_win.get(mat_id)[1])))
                loser_found = True
    
    loser_candidates.reverse()   # The absolute last one goes first in the losers' list
    scores = []
    num_losers = min(max_losers, len(loser_candidates))
    for loser in loser_candidates[:num_losers]:
        scores.append((loser, get_score(sequence, boards, loser, pulled_nums_count, verbose)))
    return scores

In [ ]:
sequence = []
boards = []
input_path = "input.txt"
get_processed_input(input_path, sequence, boards)

In [9]:
board_a, sol_a = part_a(sequence, boards)[0]
print("SOL A: [{}] is the score of {}, the winner board".format(sol_a, board_a))

SOL A: [2745] is the score of 13, the winner board


In [20]:
board_b, sol_b = part_b(sequence, boards, verbose=True)[0]
print("SOL B: [{}] is the score of {}, the winner board".format(sol_b, board_b))

13 board discarded when 3 has been pulled out
45 board discarded when 59 has been pulled out
20 board discarded when 13 has been pulled out
53 board discarded when 69 has been pulled out
6 board discarded when 83 has been pulled out
8 board discarded when 83 has been pulled out
29 board discarded when 51 has been pulled out
14 board discarded when 64 has been pulled out
75 board discarded when 64 has been pulled out
91 board discarded when 64 has been pulled out
8 board discarded when 48 has been pulled out
20 board discarded when 82 has been pulled out
45 board discarded when 82 has been pulled out
72 board discarded when 82 has been pulled out
8 board discarded when 7 has been pulled out
21 board discarded when 49 has been pulled out
26 board discarded when 8 has been pulled out
50 board discarded when 8 has been pulled out
88 board discarded when 8 has been pulled out
11 board discarded when 36 has been pulled out
59 board discarded when 36 has been pulled out
85 board discarded whe

## Example tests
### A

Winner: Third (index 2)

Score: 4512

### B

Winner (Loser): Second (index 1)

Score: 1924

In [11]:
test_sequence = [7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1]
test_boards = [
    [[22,13, 17, 11, 0],
    [8, 2, 23, 4, 24],
    [21, 9, 14, 16, 7],
    [6, 10, 3, 18, 5],
    [1, 12, 20, 15, 19]],

    [[3, 15, 0, 2, 22],
    [9, 18, 13, 17, 5],
    [19, 8, 7, 25, 23],
    [20, 11, 10, 24, 4],
    [14, 21, 16, 12, 6]],

    [[14, 21, 17, 24, 4],
    [10, 16, 15, 9, 19],
    [18,  8, 23, 26, 20],
    [22, 11, 13, 6, 5],
    [2, 0, 12, 3, 7]]
]

In [12]:
board_test_a, sol_test_a = part_a(test_sequence, test_boards)[0]
print("SOL Test A: [{}] is the score of {}, the winner board".format(sol_test_a, board_test_a))

SOL Test A: [4512] is the score of 2, the winner board


In [13]:
board_test_b, sol_test_b = part_b(test_sequence, test_boards)[0]
print("SOL Test B: [{}] is the score of {}, the winner (loser) board".format(sol_test_b, board_test_b))

SOL Test B: [1924] is the score of 1, the winner (loser) board
